In [12]:
import pandas as pd

from load_data.utils import process_all_documents, get_table_of_contents, get_paragraphs_df, read_paragraphs, read_pages_from_pdf
from load_data import load_dataframe, process_text, load_processed_data
from topic_modeling.model_optimizer import ModelOptimizer, save_data_for_app

In [ ]:
paragraphs_names= {'Executive summary': ['Executive summary'],
 "Recovery and resilience challenges: scene-setter": ['Recovery and resilience challenges: scene-setter'],
 'Objectives, structure and governance of the plan': ['Objectives, structure and governance of the plan'],
 "Summary of the assessment of the plan": ['Summary of the assessment of the plan'],
 "End paragraph": ["References", "ANNEX", "Annex"]}
data_path = "new_docs/"
results_path = "new_data_results/"

In [3]:
df = process_all_documents(data_path, paragraphs_names, data_path + 'txt_files/', "End paragraph", 'Table of contents', pages_shift = 1)
df.to_csv(data_path + 'data.csv')


In [10]:
processed_df = load_processed_data(data_path)

In [32]:
model_optimizer = ModelOptimizer(processed_df, {}, topic_numbers_range=(3, 6), lda_alpha=100)

d:\mi2\new_docs\PolicyComparison\mair-policies-comparision\topic_modeling\model_optimizer.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.data = df.loc[(df[list(column_filter)] == pd.Series(column_filter)).all(axis=1)]
100%|██████████| 3/3 [01:16<00:00, 25.60s/it]


In [33]:
save_data_for_app(model_optimizer, path=results_path)

d:\mi2\new_docs\PolicyComparison\mair-policies-comparision\topic_modeling\model_optimizer.py:83: UserWarning: country - Germany has missing rows!
  warnings.warn(f"{column} - {column_val} has missing rows!")
d:\mi2\new_docs\PolicyComparison\mair-policies-comparision\topic_modeling\model_optimizer.py:83: UserWarning: country - Romania has missing rows!
  warnings.warn(f"{column} - {column_val} has missing rows!")
d:\mi2\new_docs\PolicyComparison\mair-policies-comparision\topic_modeling\model_optimizer.py:83: UserWarning: country - Germany has missing rows!
  warnings.warn(f"{column} - {column_val} has missing rows!")
d:\mi2\new_docs\PolicyComparison\mair-policies-comparision\topic_modeling\model_optimizer.py:83: UserWarning: country - Romania has missing rows!
  warnings.warn(f"{column} - {column_val} has missing rows!")
c:\Users\artur\AppData\Local\pypoetry\Cache\virtualenvs\mair-policies-comparision-7xeuXJvh-py3.10\lib\site-packages\sklearn\manifold\_t_sne.py:991: FutureWarning: The P

[t-SNE] Computing 18 nearest neighbors...
[t-SNE] Indexed 19 samples in 0.000s...
[t-SNE] Computed neighbors for 19 samples in 0.008s...
[t-SNE] Computed conditional probabilities for sample 19 / 19
[t-SNE] Mean sigma: 1125899906842624.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 41.677284
[t-SNE] KL divergence after 850 iterations: 0.394115


d:\mi2\new_docs\PolicyComparison\mair-policies-comparision\topic_modeling\model_optimizer.py:83: UserWarning: country - Germany has missing rows!
  warnings.warn(f"{column} - {column_val} has missing rows!")
d:\mi2\new_docs\PolicyComparison\mair-policies-comparision\topic_modeling\model_optimizer.py:83: UserWarning: country - Romania has missing rows!
  warnings.warn(f"{column} - {column_val} has missing rows!")


In [34]:
from sentence_topic_analyser import SentenceTopicAnalyser

In [35]:
sentence_topic_analyser = SentenceTopicAnalyser(model_optimizer)
sentences_processed = sentence_topic_analyser.process_documents(model_optimizer.data.groupby("country")['tokens'].sum())
country_sentence_dict = dict(zip(model_optimizer.data.country.unique(), sentences_processed))

In [38]:
import json
import numpy as np


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

json.dump(country_sentence_dict, open(results_path + "sentences.json", 'w'), cls=NpEncoder)
